# Using AI Verde
## required packages

In [2]:
import os
from langchain_openai import ChatOpenAI
from docling.document_converter import DocumentConverter

# Step 0

In order to gain access to AI Verde at chat.cyverse.ai, email verde-support@cyverse.org with your information and how you'd like to use AI-Verde.

## Getting your API Key
### Visit chat.cyverse.ai and login with your netid
After logging in, click the Details button on a course you're enrolled in:

![details](./images/verde-api-key-1.png)

Use the API key to access AI-Verde's API programmatically:

![apikey](./images/verde-api-key-2.png)

Visit https://aiverde-docs.cyverse.ai/ for documentation.

## Step 1 - Set up the environment variables

In [ ]:
# Import the API keys from the environment
llm_host = os.environ.get('OPENAI_API_BASE')
api_key = os.environ.get('OPENAI_API_KEY')
model_name = "Meta-Llama-3.1-70B-Instruct-quantized"

## Step 2 - directly use LangChain ChatOpenAI

In [4]:
llm = ChatOpenAI(
    model=model_name,
    temperature=0,
    api_key=api_key,
    base_url=llm_host,
)

In [5]:
message = llm.invoke("Compose a haiku about the beauty of Tucson")
print(message.content)

Desert sunset fades
Santa Catalinas rise
Golden peaceful night


In [6]:
# show message metadata including token consumption
print(message.response_metadata)

{'token_usage': {'completion_tokens': 14, 'prompt_tokens': 44, 'total_tokens': 58, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'neuralmagic/Meta-Llama-3.1-70B-Instruct-quantized.w8a8', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}


## Demo - Translation

In [7]:
# translate a research paper in Spanish into English
# parse the PDF using Docling

pdf_path = "PLN_39_12.pdf"
converter = DocumentConverter()
result = converter.convert(pdf_path)
markdown = result.document.export_to_markdown()

prompt = "You are an expert document translator. Translate the given Markdown document into English. Do your best to correct apparent spelling and OCR issues, but do not insert words that were not in the original document."
message = [
    ("system", prompt),
    ("human", markdown[:5000]),
]
result = llm.invoke(message)
print(result.content)
#print(len(markdown))

Could not load the custom kernel for multi-scale deformable attention: /home/neddy/.cache/torch_extensions/py312_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/neddy/.cache/torch_extensions/py312_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/neddy/.cache/torch_extensions/py312_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/neddy/.cache/torch_extensions/py312_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for

## Automatic Learning for Multilingual Temporal Recognition based on TiMBL

Marcel Puchol-Blasco, Estela Saquete, Patricio Martínez-Barco
Department of Languages and Information Systems (University of Alicante)
Carretera San Vicente s/n, 03690 Alicante, Spain
{marcel,stela,patricio@dlsi.ua.es}

Abstract:
This article presents a system based on automatic learning for the recognition of temporal expressions. The system uses the TiMBL application, which is a memory-based machine learning system. The portability of this system to other new languages has a very low cost, as it does not require any language-dependent resources (only requires a tokenizer and a lexical disambiguator, although the lack of a POS tagger does not significantly affect the final system results). This system has been evaluated for three different languages: English, Spanish, and Italian. The evaluation results are quite successful for corpora with a large number of examples, while it obtains very poor results with co